In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.kalman_filters import get_spread

In [3]:
DATA_PATH = r"data"
BACKTEST =  DATA_PATH + r"\backtest.h5"
UNIVERSE = DATA_PATH + r"\etfs_stocks_universe.h5"
ASSETS = DATA_PATH + r"\assets.h5"

In [4]:
with pd.HDFStore(BACKTEST) as store:
    candidates = store['candidates']
    prices = store['prices']

In [5]:
with pd.HDFStore(UNIVERSE) as store:
    etfs = store['etfs']
    stocks = store['stocks']

In [7]:
candidates['test_end'].unique()

<DatetimeArray>
['2017-12-31 00:00:00', '2018-03-31 00:00:00', '2018-06-30 00:00:00',
 '2018-09-30 00:00:00', '2018-12-31 00:00:00', '2019-03-31 00:00:00',
 '2019-06-30 00:00:00', '2019-09-30 00:00:00', '2019-12-31 00:00:00',
 '2020-03-31 00:00:00', '2020-06-30 00:00:00', '2020-09-30 00:00:00',
 '2020-12-31 00:00:00', '2021-03-31 00:00:00', '2021-06-30 00:00:00',
 '2021-09-30 00:00:00', '2021-12-31 00:00:00', '2022-03-31 00:00:00',
 '2022-06-30 00:00:00', '2022-09-30 00:00:00', '2022-12-31 00:00:00',
 '2023-03-31 00:00:00', '2023-06-30 00:00:00', '2023-09-30 00:00:00',
 '2023-12-31 00:00:00']
Length: 25, dtype: datetime64[ns]

In [110]:
# candidates_sub = candidates[(candidates['test_end'] >= '2022-12-31') & (candidates['test_end'] < '2023-12-31')].sort_index()
candidates_sub = candidates
candidates_sub = candidates_sub.sort_values(['test_end','eg']).groupby(['test_end']).head(10)
top_pairs = candidates_sub.drop(['eg'], axis = 1)

In [111]:
pairs, half_lives = get_spread(top_pairs, pd.concat([etfs, stocks], axis = 1))
pairs_df = pd.concat(pairs)

In [116]:
def get_trades(data):
    pair_trades = []
    for i, ((period, s1, s2), pair) in enumerate(data.groupby(['period','s1','s2']), 1):
        first3m = pair.first('3M').index
        last3m = pair.last('3M').index

        entry = ((pair['z_score'].abs() > 2) & (np.sign(pair['z_score'].shift().fillna(method = 'bfill')) == np.sign(pair['z_score'])))
        entry = (entry.shift() != entry) * np.sign(pair['z_score']).fillna(0).astype(int) - 2
        exit = (np.sign(pair['z_score'].shift().fillna(method = 'bfill')) != np.sign(pair['z_score'])).astype(int) - 1

        trades = pd.concat((entry[entry != -2], exit[exit == 0])).to_frame('side').sort_values(['date','side']).squeeze()
        if not isinstance(trades, pd.Series):
            continue
        
        try:
            trades.loc[trades < 0] += 2
        except:
            break
        
        trades = trades[trades.abs().shift() != trades.abs()]
        window = trades.loc[first3m.min():first3m.max()]
        extra = trades.loc[last3m.min():last3m.max()]
        n = len(trades)

        if window.iloc[0] == 0:
            if n > 1:
                print('shift')
                window = window.iloc[1:]
        if window.iloc[-1] != 0:
            extra_exits = extra[extra == 0].head(1)
            if extra_exits.empty:
                continue
            else:
                window = pd.concat((window, extra_exits))

        trades = pair[['s1', 's2', 'hedge_ratio', 'period', 'pair']].join(window.to_frame('side'), how = 'right')
        trades.loc[trades.side == 0, 'hedge_ratio'] = np.nan
        trades.hedge_ratio = trades.hedge_ratio.ffill()
        pair_trades.append(trades)
    return pd.concat(pair_trades)

In [117]:
pair_trades = get_trades(pairs_df)

In [95]:
pair_trades['hedge_ratio'].max()

36.242130574386685

In [118]:
pair_trades[(pair_trades['s1'] == 'IBB.US') & (pair_trades['s2'] == 'META.US') ]

,s1,s2,hedge_ratio,period,pair,side
date,,,,,,
2018-01-02,IBB.US,META.US,-0.591869,1,9,-1
2018-01-09,IBB.US,META.US,-0.591869,1,9,0
2018-01-12,IBB.US,META.US,-0.583952,1,9,1
2018-01-19,IBB.US,META.US,-0.583952,1,9,0
2018-03-02,IBB.US,META.US,-0.597353,1,9,1
2018-03-07,IBB.US,META.US,-0.597353,1,9,0


In [119]:
pair_trades.to_hdf('data/backtest.h5','test_mult3/trades')